In [ ]:
#TODO
segments_amount={"rnai60":4,"wt":6,"rnai60_d":4,"wt_d":6,"pole":1,"pole_segments":list(range(3,8))}

In [ ]:
import pickle
import numpy as np
from sklearn.linear_model import LinearRegression

def get_file(name,
            gain_ratio= 0.5,
            active_gain= 0.5,
            decay= 0.15,
            pin_decay= 0.05,
            pin_production_1= 1.0,
            pin_production_2=0.02,
            dt= 0.01,):
    failed = True
    ex =None
    errors=[]
    for d in [0.,0.001,-0.001,0.002,-0.002,]:
        try:
            dormant_gain = round(gain_ratio*active_gain,3)+d
            filename =  f"./{name}/active_gain {active_gain:.3f} dormant_gain {dormant_gain:.3f} decay {decay:.3f} pin_decay {pin_decay:.3f} pin_production_1 {pin_production_1:.3f} pin_production_2 {pin_production_2:.2}"

            with open(filename,"rb") as file:
                return pickle.load(file)
        except Exception as e:
            errors.append(str(e))
            ex=e
    for error in errors:
        print(error)
    raise ex
            

def get_file2(name,
            gain_ratio= 0.5,
            active_gain= 0.5,
            decay= 0.15,
            pin_decay= 0.05,
            pin_production_1= 1.0,
            pin_production_2=0.02,
            dt= 0.01,):
    dormant_gain = gain_ratio*active_gain
    filename =  f"./v2/{name}/active_gain {active_gain:.3f} dormant_gain {dormant_gain:.3f} decay {decay:.3f} pin_decay {pin_decay:.3f} pin_production_1 {pin_production_1:.3f} pin_production_2 {pin_production_2:.2}"
    #return filename
    with open(filename,"rb") as file:
        return pickle.load(file)
from math import ceil

def get_values_average(datas,index,name):
    X = []
    Y = []
    Y2 = [[] for i in range(ceil(max([len(data) for data in datas])/segments_amount[name]))]
    for data in datas:
        start=0
        for i in range(segments_amount[name]):
            if data[start][0]<data[i][0]:
                start=i
        for i in range(0,len(data),segments_amount[name]):
            Y.append(data[i][0])
            X.append(i)
            Y2[i//segments_amount[name]].append(data[i][0])
    Y2 = [sum(x)/len(x) for x in Y2]
    X2 = list(range(0,len(data),segments_amount[name]))
    return X,Y,X2,Y2

def calculate_regression(X,Y):
    x=np.array(X).reshape((-1, 1))
    y=np.array(Y)
    model = LinearRegression().fit(x, y)
    r_sq = model.score(x, y)
    y_pred = model.predict(x)
    return model.intercept_,model.coef_[0],x,y_pred

def strip_avg(X2,Y2,ratio=0.5,strip_left=True,strip_right=True):
    X,Y=X2[:],Y2[:]
    avg_slope=abs((Y2[0]-Y2[-1])/len(X2))
    if abs(Y2[0]-Y2[1])<avg_slope*ratio and strip_left:
        X,Y=X2[1:],Y2[1:]
    if abs(Y2[-2]-Y2[-1])<avg_slope*ratio and strip_right:
        X,Y=X2[:-1],Y2[:-1]
    return X,Y

def strip_avg_full(X2,Y2,ratio=0.5,strip_left=True,strip_right=True):
    X,Y=strip_avg(X2,Y2,ratio,strip_left,strip_right)
    old_l=len(X2)
    while len(X)!=old_l:
        old_l=len(X)
        X,Y=strip_avg(X,Y)
    return X,Y

from matplotlib import cm
colormap = cm.get_cmap("Set3")
def get_color(i):
    return (colormap((i%colormap.N)/12))

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="",xlabel="",ylabel="",label_fontsize=10, **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))

    ax.set_xlabel(xlabel,fontsize=label_fontsize)
    ax.set_ylabel(ylabel,fontsize=label_fontsize)
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("white","black"),color_range=(0,),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            ci=0
            for r in color_range:
                if data[i, j]<r:
                    break
                ci+=1
            kw.update(color=textcolors[ci])
            text = im.axes.text(j, i, valfmt(data[i, j], (i,j)), **kw)
            texts.append(text)

    return texts

In [ ]:
import matplotlib.pyplot as plt
starting_values = []
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
for gain in active_gain:
    data = get_file("pole_segments",active_gain=gain,gain_ratio=1.)[0]
    #print(data)
    starting_values.append(data[0][0])
    plt.plot([x[0] for x in data])
    #plt.plot([x[1] for x in data])
plt.show()
plt.plot(active_gain,starting_values)


In [ ]:
from matplotlib import cm
colormap = cm.get_cmap("Set3")
def get_color(i):
    return (colormap((i%colormap.N)/12))

In [ ]:
plt.boxplot([[1,1,1,4,4,5],[1,2,3,4,5]],positions=[0,5])
plt.show()

In [ ]:
from math import ceil
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[10,10]
starting_values = []
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
temp=None
names=["rnai60_d","wt_d"]
nice_names = ["RNAi60 decapitated","Wild Type decapitated"]
handles=[]
colors_idx = [5,0]
gain=active_gain[5]
segments_amount
fig= plt.figure(figsize=(12,6),constrained_layout=True)
ax = fig.subplots(nrows=1, ncols=2)
start=1
def filter_accepted(start,step,X,Y):
    accepted = set(range(start,max(X)+1,step))
    X0,Y0=[],[]
    for x,y in zip(X,Y):
        if x in accepted:
            X0.append(x)
            Y0.append(y)
    return X0,Y0

for i,name in enumerate(names):
    ii=colors_idx[i]
    datas = get_file(name,
                    active_gain=0.7,
                    gain_ratio=0.17,
                    decay=0.155,
                    pin_decay=0.05,
                    pin_production_1=1.,
                    pin_production_2=0.06,)
    print(name)
    print("")
    X,Y,X2,Y2=get_values_average(datas,0,name)
    X=[int(x/segments_amount[name]) for x in X]
    X2=list(map(lambda x: int(x/segments_amount[name]),X2))
    step=4
    X,Y=filter_accepted(start,step,X,Y)
    X2,Y2=filter_accepted(start,step,X2,Y2)
     
    Y2=np.array(Y2)*2000
    Y=np.array(Y)*2000
    positions = sorted(map(int,set(X)))
    boxes = [[] for i in positions]
    X_B=[x+18 for x in X]
    for x,y in zip(X,Y):
        boxes[positions.index(int(x))].append(y)
    ax[i].scatter(X,Y,s=10,color=get_color(ii),alpha=0.5)
    #handles=[ax[i].plot(X2,Y2,color=get_color(ii),label=f"{nice_names[i]}")[0]]
    #ax[i].plot(X2,Y2,color=get_color(ii))
    a,b,X3,Y3=calculate_regression(X,Y)
    ax[i].plot(X3,Y3,color=get_color(ii),alpha=0.7)
    #plt.plot([x[1] for x in data])
    ax[i].set_xlabel("internode index",fontsize=13)
    ax[i].set_ylabel("auxin value",fontsize=13)
    bp = ax[i].boxplot(boxes,positions=positions,widths=[2]*len(positions))    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color=get_color(ii))
    ax[i].set_title(nice_names[i],fontsize=15)
    ax[i].set_ylim(0,1500)
    #ax[i].legend(handles=handles)
plt.show()
#plt.plot(active_gain,starting_values)

In [ ]:
from math import ceil
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[10,10]
starting_values = []
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
temp=None
names=["rnai60_d","wt_d"]
nice_names = ["RNAi60 decapitated","Wild Type decapitated"]
handles=[]
colors_idx = [5,0]
gain=active_gain[5]
segments_amount
fig= plt.figure(figsize=(12,6),constrained_layout=True)
ax = fig.subplots(nrows=1, ncols=2)
start=1
def filter_accepted(start,step,X,Y):
    accepted = set(range(start,max(X)+1,step))
    X0,Y0=[],[]
    for x,y in zip(X,Y):
        if x in accepted:
            X0.append(x)
            Y0.append(y)
    return X0,Y0

import csv

for i,name in enumerate(names):
    ii=colors_idx[i]
    datas = get_file(name,
                    active_gain=0.7,
                    gain_ratio=0.17,
                    decay=0.155,
                    pin_decay=0.05,
                    pin_production_1=1.,
                    pin_production_2=0.06,)
    print(name)
    print("")
    X,Y,X2,Y2=get_values_average(datas,0,name)
    X=[int(x/segments_amount[name]) for x in X]
    X2=list(map(lambda x: int(x/segments_amount[name]),X2))
    start=1
    step=4
    X,Y=filter_accepted(start,step,X,Y)
    X2,Y2=filter_accepted(start,step,X2,Y2)
     
    Y2=np.array(Y2)*2000
    Y=np.array(Y)*2000
    positions = sorted(map(int,set(X)))
    boxes = [[] for i in positions]
    print(X)
    X=[x+18 for x in X]
    with open(f'{nice_names[i]}.tsv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f,delimiter="\t")
        writer.writerow(["internode","auxin level"])
        for row in zip(X,Y):
            writer.writerow(row)
            

In [ ]:
from math import ceil
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[10,10]
starting_values = []
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
temp=None
names=["rnai60","wt"]
nice_names = ["RNAi60","Wild Type"]
handles=[]
colors_idx = [5,0]
gain=active_gain[5]
segments_amount
fig= plt.figure(figsize=(12,6),constrained_layout=True)
ax = fig.subplots(nrows=1, ncols=2)
start=1
def filter_accepted(start,step,X,Y):
    accepted = set(range(start,max(X)+1,step))
    X0,Y0=[],[]
    for x,y in zip(X,Y):
        if x in accepted:
            X0.append(x)
            Y0.append(y)
    return X0,Y0

import csv

for i,name in enumerate(names):
    ii=colors_idx[i]
    datas = get_file(name,
                    active_gain=0.7,
                    gain_ratio=0.17,
                    decay=0.155,
                    pin_decay=0.05,
                    pin_production_1=1.,
                    pin_production_2=0.06,)
    print(name)
    print("")
    X,Y,X2,Y2=get_values_average(datas,0,name)
    X=[int(x/segments_amount[name]) for x in X]
    X2=list(map(lambda x: int(x/segments_amount[name]),X2))
    start=1
    step=4
    X,Y=filter_accepted(start,step,X,Y)
    X2,Y2=filter_accepted(start,step,X2,Y2)
     
    Y2=np.array(Y2)*2000
    Y=np.array(Y)*2000
    positions = sorted(map(int,set(X)))
    boxes = [[] for i in positions]
    print(X)
    with open(f'{nice_names[i]}.tsv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f,delimiter="\t")
        writer.writerow(["internode","auxin level"])
        for row in zip(X,Y):
            writer.writerow(row)
            

In [ ]:
from math import ceil
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[10,10]
starting_values = []
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
temp=None
names=["rnai60","wt","pole"]
handles=[]
for name in names:
    for ii,gain in enumerate(active_gain):
        datas = get_file(name,active_gain=gain,gain_ratio=0.5)
        X,Y,X2,Y2=get_values_average(datas,0,name)
        if name == "wt" and gain==0.5:
            temp=X2,Y2
        plt.scatter(X,Y,s=0.8,color=get_color(ii))
        handles.append(plt.plot(X2,Y2,color=get_color(ii),label=f"auxin production value {gain}")[0])
        plt.plot(X2,Y2,color=get_color(ii))
        a,b,X3,Y3=calculate_regression(*strip_avg_full(X2,Y2))
        plt.plot(X3,Y3,color=get_color(ii),alpha=0.3)
        #plt.plot([x[1] for x in data])
    plt.xlabel("segment index")
    plt.ylabel("auxin value")
    plt.legend(handles=handles)
    plt.show()
#plt.plot(active_gain,starting_values)


In [ ]:

from math import ceil
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[10,10]
starting_values = []
gain_ratios = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
names=["rnai60","wt","pole"]
for gain in gain_ratios:
    print(f"gain {gain}")
    for name in names:
        handles=[]
        for ii,gain_ratio in enumerate(gain_ratios):
            datas = get_file(name,active_gain=gain,gain_ratio=gain_ratio)
            X,Y,X2,Y2=get_values_average(datas,0,name)
            # X = []
            # Y = []
            # Y2 = [[] for i in range(ceil(max([len(data) for data in datas])/segments_amount[name]))]
            # for data in datas:
            #     for i in range(0,len(data),segments_amount[name]):
            #         Y.append(data[i][0])
            #         X.append(i)
            #         Y2[i//segments_amount[name]].append(data[i][0])
            # Y2 = [sum(x)/len(x) for x in Y2]
            # X2 = list(range(0,len(data),segments_amount[name]))
            plt.scatter(X,Y,s=0.8,color=get_color(ii))
            handles.append(plt.plot(X2,Y2,color=get_color(ii),label=f"dormant production ratio {gain_ratio}")[0])
            a,b,X3,Y3=calculate_regression(*strip_avg_full(X2,Y2,ratio=0.8))
            plt.plot(X3,Y3,color=get_color(ii),alpha=0.3)
            print(a,b)
            #plt.plot([x[1] for x in data])
        plt.title(name)
        #plt.legend([f"dormant production ratio {g}" for g in gain_ratios])
        plt.legend(handles=handles)
        plt.show()
    #plt.plot(active_gain,starting_values)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

def get_abs_range(arr):
    return max(abs(np.max(arr)),abs(np.min(arr)))

plt.rcParams['figure.figsize']=[10,10]
starting_values = []
gain_ratios = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,]
names=["rnai60","wt"]
for name in names:
    values_a = np.zeros((len(active_gain),len(gain_ratios)))
    values_b = np.zeros((len(active_gain),len(gain_ratios)))
    values_c = np.zeros((len(active_gain),len(gain_ratios)))
    for x,gain in enumerate(active_gain):
        for y,gain_ratio in enumerate(gain_ratios):
            datas = get_file(name,active_gain=gain,gain_ratio=gain_ratio)
            X,Y,X2,Y2=get_values_average(datas,0,name)
            b,a,X3,Y3=calculate_regression(*strip_avg_full(X2,Y2,ratio=0.7))
            values_a[x,y]=a*100
            values_b[x,y]=b
            values_c[x,y]=Y2[-1]
    # plt.imshow(values_a)
    # plt.colorbar()
    # plt.show()
    # plt.title(f"{name} b")
    # plt.imshow(values_b)
    # plt.colorbar()
    # plt.show()
    
    vmax = get_abs_range(values_a)
    fig, ax = plt.subplots()
    ax.set_title(f"{name} slope")
    im, cbar = heatmap(values_a, active_gain,gain_ratios, ax=ax,
                    cmap="magma", cbarlabel="",xlabel="gain ratio",ylabel="gain value",vmin=-vmax, vmax=vmax)
    texts = annotate_heatmap(im, valfmt="{x:.1f} t",textcolors=("white","black"))
    fig.tight_layout()
    plt.show()
    
    vmax = get_abs_range(values_c)
    fig, ax = plt.subplots()
    ax.set_title(f"{name} final value")
    im, cbar = heatmap(values_c, active_gain,gain_ratios, ax=ax,
                    cmap="magma", cbarlabel="",xlabel="gain ratio",ylabel="gain value",vmin=0, vmax=vmax)
    texts = annotate_heatmap(im, valfmt="{x:.3f} t",textcolors=("white","black"))
    fig.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import math

def get_abs_range(arr):
    return max(abs(np.max(arr)),abs(np.min(arr)))

plt.rcParams['figure.figsize']=[10,10]
starting_values = []
gain_ratios = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

def formatter_creator(values1,values2):
    def format_function(x,pos):
        d=max(0,math.floor(math.log10(abs(x)))+1)
        return f"{{:.{max(0,3-d)}f}}".format(x)#+f"\n{values1[pos[0]]}\n{values2[pos[1]]}"
    return format_function

def create_heatmap(ax,values,values1,values2,name1,name2,label,neg_scale=True):
    vmax = min(3000,get_abs_range(values))
    ax.set_title(label)
    im, cbar = heatmap(values, values1,values2, ax=ax,
                    cmap="bwr", cbarlabel="",xlabel=name2,ylabel=name1,vmin=-vmax, vmax=vmax)
    

    texts = annotate_heatmap(im, valfmt=formatter_creator(values1,values2),textcolors=("white","blue","red","white"),color_range=[-vmax/2,0,vmax/2])
    #fig.tight_layout()
    #plt.show()
def draw_heatmaps(name1,name2,values1,values2,save=True,extra_parameters={},test2=False,filename=None):
    names=["rnai60","wt","pole"]
    names=["rnai60","wt"]
    subplots = 3
    fig= plt.figure(figsize=(subplots*10,len(names)*10),constrained_layout=True)
    fig.suptitle(f"{name1} vs {name2}",fontsize='xx-large')
    subfigs = fig.subfigures(nrows=len(names), ncols=1)
    
    pretty_names=["Kantarelli","Wild Type","Pole"]

    multiplier_coefficient = 1500
    for i,name in enumerate(names):
        values_a = np.zeros((len(values1),len(values2)))
        values_b = np.zeros((len(values1),len(values2)))
        values_c = np.zeros((len(values1),len(values2)))
        values_d = np.zeros((len(values1),len(values2)))
        values_e = np.zeros((len(values1),len(values2)))
        for x,v1 in enumerate(values1):
            for y,v2 in enumerate(values2):
                datas = get_file(name,**{name1:v1,name2:v2},**extra_parameters)
                X,Y,X2,Y2=get_values_average(datas,0,name)
                X3,Y3 = strip_avg_full(X2,Y2,ratio=0.65)
                b,a,X4,Y4=calculate_regression(X3,Y3)
                values_a[x,y]=a*multiplier_coefficient
                values_b[x,y]=b*multiplier_coefficient
                values_c[x,y]=(Y2[-1])*multiplier_coefficient
                values_d[x,y]=(Y2[0])*multiplier_coefficient
                values_e[x,y]=len(Y3)
        # plt.imshow(values_a)
        # plt.colorbar()
        # plt.show()
        # plt.title(f"{name} b")
        # plt.imshow(values_b)
        # plt.colorbar()
        # plt.show()
        subfigs[i].suptitle(pretty_names[i],fontsize='x-large',color="Red")
        ax = subfigs[i].subplots(nrows=1, ncols=subplots)
        create_heatmap(ax[0],values_a,values1,values2,name1,name2,f"{name} slope")
        
        create_heatmap(ax[1],values_d,values1,values2,name1,name2,f"{name} starting value")

        create_heatmap(ax[2],values_c,values1,values2,name1,name2,f"{name} final value",neg_scale=False)
        #create_heatmap(ax[3],values_e,values1,values2,name1,name2,f"{name} len",neg_scale=False)
    if save:
        if filename is None:
            filename=f"{name1} vs {name2}.png"
        plt.savefig(filename)
    else:
        plt.show()


        
#draw_heatmaps("active_gain","decay",[0.2, 0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7,
#       1.8, 1.9, 2. ],[0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ])


In [ ]:
def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("white","black"),color_range=(0,),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            ci=0
            for r in color_range:
                if data[i, j]<r:
                    break
                ci+=1
            kw.update(color=textcolors[ci])
            text = im.axes.text(j, i, valfmt(data[i, j], (i,j)), **kw)
            texts.append(text)

    return texts

def create_heatmap2(ax,values,values1,values2,name1,name2,label,neg_scale=True):
    vmax = min(3000,get_abs_range(values))
    #ax.set_title(label,backgroundcolor=(0.7,0.7,0.7),color="white", fontsize=20)
    ax.set_title(f"{label}\n",fontsize=40)
    im, cbar = heatmap(values, values1,values2, ax=ax,
                    cmap="bwr", cbarlabel="",xlabel=name2,ylabel=name1,vmin=-vmax, vmax=vmax,label_fontsize=30)
    

    texts = annotate_heatmap(im, valfmt=formatter_creator(values1,values2),textcolors=("white","blue","red","white"),color_range=[-vmax/2,0,vmax/3],fontsize=11)




def draw_heatmaps2(name1,name2,values1,values2,save=True,extra_parameters={},test2=False,filename=None, maintitle=None):
    names=["rnai60","wt","pole"]
    names=["rnai60","wt"]
    subplots = 1
    fig= plt.figure(figsize=(len(names)*8,subplots*10),constrained_layout=True)
    if maintitle is None:
        maintitle=f"{name1} vs {name2}\n\n\n"
    #fig.suptitle(maintitle,fontsize='xx-large',backgroundcolor=(0.7,0.7,0.7),color="white")
    ax = fig.subplots(nrows=1, ncols=2)
    
    pretty_names=["RNAi60","Wild Type","Pole"]
    print("sss")
    multiplier_coefficient = 1500
    for i,name in enumerate(names):
        values_a = np.zeros((len(values1),len(values2)))
        values_b = np.zeros((len(values1),len(values2)))
        values_c = np.zeros((len(values1),len(values2)))
        values_d = np.zeros((len(values1),len(values2)))
        values_e = np.zeros((len(values1),len(values2)))
        for x,v1 in enumerate(values1):
            for y,v2 in enumerate(values2):
                datas = get_file(name,**{name1:v1,name2:v2},**extra_parameters)
                X,Y,X2,Y2=get_values_average(datas,0,name)
                X3,Y3 = strip_avg_full(X2,Y2,ratio=0.65)
                b,a,X4,Y4=calculate_regression(X3,Y3)
                values_a[x,y]=a*multiplier_coefficient
                values_b[x,y]=b*multiplier_coefficient
                values_c[x,y]=(Y2[-1])*multiplier_coefficient
                values_d[x,y]=(Y2[0])*multiplier_coefficient
                values_e[x,y]=len(Y3)
        # plt.imshow(values_a)
        # plt.colorbar()
        # plt.show()
        # plt.title(f"{name} b")
        # plt.imshow(values_b)
        # plt.colorbar()
        # plt.show()
        create_heatmap2(ax[i],values_a,values1,values2,"auxin production p","auxin decay d",f"{pretty_names[i]}")
        #create_heatmap(ax[3],values_e,values1,values2,name1,name2,f"{name} len",neg_scale=False)
    if save:
        if filename is None:
            filename=f"{name1} vs {name2}.png"
        plt.savefig(filename)
    else:
        plt.show()

decay= [0.125, 0.13, 0.135, 0.14 , 0.145, 0.15 , 0.155, 0.16 , 0.165]
active_gain= [0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.56, 0.58, 0.6 ]
active_gain= [0.36, 0.38, 0.4 , 0.42, 0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 ]

decay=      [0.055, 0.075, 0.095, 0.115, 0.135, 0.155, 0.175, 0.195, 0.215, 0.235, 0.255]
active_gain=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
print("active_gain vs decay")
draw_heatmaps2("active_gain","decay",
               active_gain,decay,save=False,extra_parameters=dict(
                    gain_ratio=0.17,
                    pin_decay=0.05,
                    pin_production_1=1.,
                    pin_production_2=0.06,))
        
#draw_heatmaps("active_gain","decay",[0.2, 0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7,
#       1.8, 1.9, 2. ],[0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ])


In [ ]:
from math import ceil
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[10,10]
starting_values = []
gain_ratios = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
names=["rnai60","wt","pole"]
for gain in gain_ratios:
    print(f"gain {gain}")
    for name in names:
        for gain_ratio in gain_ratios:
            datas = get_file(name,active_gain=0.5,gain_ratio=0.5,pin_production)
            X = []
            Y = []
            Y2 = [[] for i in range(ceil(max([len(data) for data in datas])/segments_amount[name]))]
            for data in datas:
                for i in range(0,len(data),segments_amount[name]):
                    Y.append(data[i][0])
                    X.append(i)
                    Y2[i//segments_amount[name]].append(data[i][0])
            Y2 = [sum(x)/len(x) for x in Y2]
            X2 = list(range(0,len(data),segments_amount[name]))
            plt.scatter(X,Y,s=0.8)
            plt.plot(X2,Y2)
            #plt.plot([x[1] for x in data])
        plt.title(name)
        plt.legend([f"dormant production value {g}" for g in active_gain])
        plt.show()
    #plt.plot(active_gain,starting_values)


In [ ]:
from os import listdir
from os.path import isfile, join
data = {x:set() for x in listdir("wt")[0].split()[::2]}
tuples = set()
names = ""
for name in listdir("rnai60"):
    splitted = name.split()
    for a,b in zip(splitted[::2],splitted[1::2]):
        data[a].add(b)
    tuples.add(tuple(splitted[1::2]))
    names=splitted[0::2]

In [ ]:
print(names)

In [ ]:
def show_values(a,b):
    fig = plt.figure(figsize=(16,16))
    plt.scatter([float(x[a]) for x in tuples],[float(x[b]) for x in tuples])
    for x,y in set(zip([float(x[a]) for x in tuples],[float(x[b]) for x in tuples])):
        plt.annotate(f"|{x},{y}|",(x,y),fontsize=7)
    plt.title(" vs ".join([names[a],names[b]]))
    plt.show()
for a in range(0,5):
    for b in range(a+1,6):
        # if names[a]!='decay':
        #     continue
        show_values(a,b)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
def calculate_regression(X,Y):
    x=np.array(X).reshape((-1, 1))
    y=np.array(Y)
    model = LinearRegression().fit(x, y)
    r_sq = model.score(x, y)
    print(f"coefficient of determination: {r_sq}")
    plt.figure(figsize=(3,3))
    y_pred = model.predict(x)
    plt.plot(x,y)
    plt.plot(x,y_pred)
    return model.intercept_,model.coef_[0],x,y_pred
def strip_avg(X2,Y2,ratio=0.5,strip_left=True,strip_right=True):
    X,Y=X2[:],Y2[:]
    avg_slope=abs((Y2[0]-Y2[-1])/len(X2))
    if abs(Y2[0]-Y2[1])<avg_slope*ratio and strip_left:
        X,Y=X2[1:],Y2[1:]
    if abs(Y2[-2]-Y2[-1])<avg_slope*ratio and strip_right:
        X,Y=X2[:-1],Y2[:-1]
    return X,Y

def strip_avg_full(X2,Y2,ratio=0.5,strip_left=True,strip_right=True):
    X,Y=strip_avg(X2,Y2,ratio,strip_left,strip_right)
    old_l=len(X2)
    while len(X)!=old_l:
        old_l=len(X)
        X,Y=strip_avg(X,Y)
    return X,Y

X2,Y2= temp
X,Y=strip_avg_full(X2,Y2)
model=calculate_regression(X,Y)


In [ ]:

#gain_ratios vs active_gain
names=["rnai60","wt","pole"]
gain_ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.]
active_gain = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
print("gain_ratio vs active_gain")
draw_heatmaps("gain_ratio","active_gain",gain_ratios,active_gain)

#active_gain vs decay
print("active_gain vs decay")
draw_heatmaps("active_gain","decay",[0.1, 0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7,
       1.8, 1.9, 2. ],[0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ])

#decay vs pin_decay
pin_decay=[0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.15, 0.2, 0.25]
decay = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3]
print("pin_decay vs decay")
draw_heatmaps("pin_decay","decay",pin_decay,decay)

#decay vs pin_production 
decay = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3]
pin_production_1 = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
print("decay vs pin_production_1")
draw_heatmaps("decay","pin_production_1",decay,pin_production_1)

#dormant_gain vs decay
# decay = [0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ]
# dormant_gain = [0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75,    0.8 , 0.85, 0.9 , 0.95, 1.  ]
# print("dormant_gain vs decay")
# draw_heatmaps("decay","activr",decay,dormant_gain)

#pin_decay vs pin_production
# pin_decay=[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
#        0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
#        0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
#        0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
#        0.45, 0.46, 0.47, 0.48, 0.49, 0.5 ]
# pin_production_1 = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
# print("pin_decay vs pin_production")
# draw_heatmaps("pin_decay","pin_production_1",pin_decay,pin_production_1)     
#pin_production_1 vs pin_production_2
pin_production_1 = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3,1.4,1.5,1.6,1.7]
pin_production_2 = [0.01,0.02,0.03,0.04,0.05]
print("pin_production_1 vs pin_production_2")
draw_heatmaps("pin_production_1","pin_production_2",pin_production_1,pin_production_2)



In [ ]:
active_gain= [0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56]
gain_ratios= [0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61]
pin_decay= [0.04, 0.05]
decay= [0.15, 0.175, 0.2]

names=["rnai60","wt","pole"]
print("gain_ratio vs active_gain")
for pd in pin_decay:
    for d in decay:
        print(f"pin_decay {pd},decay {d}")
        draw_heatmaps("gain_ratio","active_gain",gain_ratios,active_gain,save=False,extra_parameters={"pin_decay":pd,"decay":d})

In [ ]:
pin_decay= [0.04,0.05]
decay= [0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16]
active_gain= [0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5]

print("active_gain vs decay")
for pd in pin_decay:
    print(f"pin_decay {pd}")
    draw_heatmaps("active_gain","decay",active_gain,decay,save=True,extra_parameters={"pin_decay":pd},filename=f"precise test/active_gain vs decay pin_decay {pd}.png")